In [1]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas/')
import metatlas.datastructures.metatlas_objects as metob
# from metatlas.interfaces.compounds import structure_cleaning as metclean
# from metatlas.tools import cheminfo
from metatlas.interfaces.compounds import structure_cleaning as cleaning

# import sys
# sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Descriptors import ExactMolWt
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import numpy as np
import pubchempy as pcp

import pandas as pd

In [2]:
cpd_df = pd.read_csv('/global/cfs/cdirs/metatlas/projects/unique_compounds.csv.gz',usecols=['name','inchi','inchi_key'])
print(cpd_df.shape)
cpd_df.head()

(217668, 3)


,inchi,inchi_key,name
0,InChI=1S/C23H24O5/c1-23(2)12-11-16-18(28-23)14...,CDFJITYOZNLONM-UYRXBGFRSA-N,Praecanson A
1,InChI=1S/C21H19N3OS/c1-13-8-9-16(10-14(13)2)22...,JZEFBZMLAJEBMC-UHFFFAOYSA-N,AC1LKV71
2,InChI=1S/C20H17ClN2O4/c1-25-15-8-6-14(7-9-15)1...,BQGDCUBBCXAHON-UHFFFAOYSA-N,AC1LKV6I
3,InChI=1S/C23H47NO2/c1-3-4-5-13-16-20(2)17-14-1...,MXSVVKSZJQKNBC-AVAPBHFXSA-N,Penazetidine A
4,InChI=1S/C2N2/c3-1-2-4,JMANVNJQNLATNU-UHFFFAOYSA-N,CYANOGEN


In [21]:
df.shape

(176459, 20)

In [22]:
# df = pd.read_csv('/global/cfs/cdirs/metatlas/projects/spectral_libraries/deduplicated_merged_library/dm_msms_refs.tab',sep='\t')
# df

In [3]:
df = pd.read_csv('/global/cfs/cdirs/metatlas/projects/spectral_libraries/deduplicated_merged_library/dm_msms_refs.tab',sep='\t',usecols=['name','inchi_key','inchi','smiles','precursor_mz','spectrum','database','polarity'])


In [4]:
df.drop_duplicates('inchi_key',inplace=True)
df = df[~df['inchi_key'].isin(cpd_df['inchi_key'])]
# df.set_index('inchi_key',inplace=True)
sum(pd.isna(df['inchi']))

0

In [11]:
df.shape

(11776, 8)

In [10]:
sum(df['name'].apply(len)<1)

0

In [13]:
metob.Compound()

{'chebi_id': '',
 'chebi_url': '',
 'creation_time': '2023-01-12T12:55:19',
 'description': 'No description',
 'formula': '',
 'head_id': '951f04a05aa9461b97d97f7beac9ddea',
 'hmdb_id': '',
 'hmdb_url': '',
 'img_abc_id': '',
 'inchi': '',
 'inchi_key': '',
 'iupac_name': '',
 'kegg_id': '',
 'kegg_url': '',
 'last_modified': '2023-01-12T12:55:19',
 'lipidmaps_id': '',
 'lipidmaps_url': '',
 'metacyc_id': '',
 'mono_isotopic_molecular_weight': 0.0,
 'name': 'Untitled',
 'neutralized_2d_inchi': '',
 'neutralized_2d_inchi_key': '',
 'neutralized_inchi': '',
 'neutralized_inchi_key': '',
 'num_free_radicals': 0,
 'number_components': 0,
 'permanent_charge': 0,
 'prev_uid': '',
 'pubchem_compound_id': '',
 'pubchem_url': '',
 'source': '',
 'synonyms': '',
 'unique_id': '951f04a05aa9461b97d97f7beac9ddea',
 'username': 'bpb',
 'wikipedia_url': ''}

In [17]:
compounds = []
for i,row in df.iterrows():
    mol = Chem.MolFromInchi(row['inchi'])
    computed_inchi = Chem.MolToInchi(mol)
    computed_ik = Chem.InchiToInchiKey(computed_inchi)
    if computed_ik!=row['inchi_key']:
        print(i)
        # This should never happen, but good to check
    comp = metob.Compound()
    comp.name = row['name']
    comp.inchi = row['inchi']
    comp.inchi_key = row['inchi_key']
    comp.formula = Chem.rdMolDescriptors.CalcMolFormula(mol)
    comp.mono_isotopic_molecular_weight = ExactMolWt(mol)
    comp.permanent_charge = Chem.GetFormalCharge(mol)
    # comp.number_components = comp.number_components or 1  # type: ignore
    # comp.num_free_radicals = comp.num_free_radicals or Chem.Descriptors.NumRadicalElectrons(norm_mol)
    compounds.append(comp)

In [19]:
metob.store(compounds)

# get new inchi for missing ones

In [5]:
iks = df.loc[pd.isna(df['inchi']),'inchi_key'].tolist()
# iks = iks[:3]
df_no_inchi = pcp.get_properties(['inchi', 'inchikey'], iks, 'inchikey', as_dataframe=True)
df_no_inchi.rename(columns={'InChI':'inchi','InChIKey':'inchi_key'},inplace=True)
# df_no_inchi.set_index('inchi_key',inplace=True)
for i,row in df_no_inchi.iterrows():
    df.loc[df['inchi_key']==row['inchi_key'],'inchi'] = row['inchi']
# df.update(df_no_inchi)#[df_no_inchi['inchi_key'].tolist(),'inchi'] = df_no_inchi['inchi']


In [6]:
sum(pd.isna(df['inchi']))

172

In [7]:
cids = []
linker = []
for ik in df.loc[pd.isna(df['inchi']),'inchi_key'].tolist():
    temp = pcp.get_cids(ik, 'inchikey', 'compound', list_return='flat')
    for t in temp:
        linker.append({'inchi_key':ik,'cid':t})
    cids.extend(temp)
    
df_no_inchi = pcp.get_properties(['inchi', 'inchikey'], cids, 'cid', as_dataframe=True,)
df_no_inchi = pd.merge(df_no_inchi,pd.DataFrame(linker),left_index=True,right_on='cid')
for i,row in df_no_inchi.iterrows():
    df.loc[df['inchi_key']==row['inchi_key'],'inchi'] = row['InChI']

In [8]:
sum(pd.isna(df['inchi']))

158

In [9]:
cids = []
linker = []
for name in df.loc[pd.isna(df['inchi']),'name'].tolist():
    temp = pcp.get_cids(name, 'name', 'substance', list_return='flat')
    for t in temp:
        linker.append({'name':name,'cid':t})
    cids.extend(temp)

df_no_inchi = pcp.get_properties(['inchi', 'inchikey'], cids, 'cid', as_dataframe=True,)
df_no_inchi = pd.merge(df_no_inchi,pd.DataFrame(linker),left_index=True,right_on='cid')
df_no_inchi.drop_duplicates('name',inplace=True)
for i,row in df_no_inchi.iterrows():
    df.loc[df['name']==row['name'],'inchi'] = row['InChI']

In [10]:
sum(pd.isna(df['inchi']))

35

In [11]:
df[pd.isna(df['inchi'])].to_csv('compounds that I can not find.csv',index=None)

In [12]:
print(df.shape)
df = df[pd.notna(df['inchi'])]
print(df.shape)

(12797, 8)
(12762, 8)


In [13]:
df['mol'] = df['inchi'].apply(Chem.MolFromInchi)
print(df.shape)
df = df[pd.notna(df['inchi'])]
print(df.shape)
n = []
for i,row in df.iterrows():
    try:
        desalted, _ = cleaning.desalt(row['mol'])
        m, _ = cleaning.NeutraliseCharges(desalted)
    except:
        print('Using original mol.  Failed for ',row)
        m = row['mol']
    n.append(m)
df['normalized_mol'] = n

(12762, 9)
(12762, 9)
Using original mol.  Failed for  database                                             GNPS-LIBRARY
name                                                   Heme B M+H
spectrum        [[51.28997, 58.88598, 59.36004, 62.96975, 64.7...
precursor_mz                                              616.188
polarity                                                 Positive
inchi_key                             RNQMHXMGXVQRMV-UHFFFAOYSA-J
inchi           InChI=1S/C34H34N4O4.Fe/c1-7-21-17(3)25-13-26-1...
smiles          C=Cc1c(C)c2cc3[n-]c(cc4nc(cc5nc(cc1[n-]2)C(C)=...
mol              <rdkit.Chem.rdchem.Mol object at 0x14a73c7186d0>
Name: 134853, dtype: object
Using original mol.  Failed for  database                                             GNPS-LIBRARY
name                                       4-N-methyllyaloside M+
spectrum        [[87.30630, 90.08881, 94.15279, 98.25227, 113....
precursor_mz                                              541.219
polarity                  

In [16]:
df['norm_inchi'] = df['normalized_mol'].apply(Chem.MolToInchi)
df['norm_ik'] = df['norm_inchi'].apply(Chem.InchiToInchiKey)

In [17]:
sum(df['norm_ik'].isin(cpd_df['inchi_key'])),df.shape

(1144, (12762, 12))

# Get the ones that need to be added to metatlas

In [18]:
idx1 = df['norm_ik'].isin(cpd_df['inchi_key'])
idx2 = pd.isna(df['name'])
df[(~idx1) & (idx2)]

,database,name,spectrum,precursor_mz,polarity,inchi_key,inchi,smiles,mol,normalized_mol,norm_inchi,norm_ik
126186,nist,NaN,"[[310.09700, 338.12790, 495.22750, 496.23030, ...",538.2336,positive,ZHEHVZXPFVXKEY-KPNMDXRMSA-N,InChI=1S/C32H31N3O5/c1-3-4-5-10-15-39-30(37)32...,NaN,<rdkit.Chem.rdchem.Mol object at 0x14a73ce30eb0>,<rdkit.Chem.rdchem.Mol object at 0x14a73cb7fb30>,InChI=1S/C32H31N3O5/c1-3-4-5-10-15-39-30(37)32...,ZHEHVZXPFVXKEY-KPNMDXRMSA-N


In [ ]:
# Check that ExactMolWt(normalized_mol) + Adduct is the precursor

# OLD STUFF BELOW HERE.  COPY-PASTE AT YOUR OWN RISK!!!!!

In [ ]:
# df.loc[pd.isna(df['inchi']),['inchi_key','name','smiles']]
df_no_inchi = pcp.get_properties(['inchi', 'inchikey'], df.loc[pd.isna(df['inchi']),'name'].tolist(), 'name', as_dataframe=True,)
df_no_inchi
# print('\n'.join([p for p in df.loc[pd.isna(df['inchi']),'name']]))

# neuatrlaze and desalt all inchi

In [ ]:
row

In [ ]:
df[idx]

In [ ]:
pd.merge(df,df_no_inchi,on='inchi_key')

In [ ]:
def flatten_inchi(mol: Chem.rdchem.Mol) -> str:
    smiles = Chem.MolToSmiles(mol).replace("@", "")
    flattened_rdkit_mol = Chem.MolFromSmiles(smiles)
    try:
        return Chem.MolToInchi(flattened_rdkit_mol)
    except Exception:  # This fails when can't kekulize mol # pylint: disable=broad-except
        logger.warning("failed to flatten a molecule")
        return ""

comp = metob.Compound()
for i,row in df_no_inchi.head(1).iterrows():
    norm_mol = cheminfo.normalize_molecule(row['mol'])
    if not comp.neutralized_inchi:
        comp.neutralized_inchi = Chem.inchi.MolToInchi(norm_mol)
    if not comp.neutralized_inchi_key:
        comp.neutralized_inchi_key = Chem.inchi.InchiToInchiKey(comp.neutralized_inchi)
    if not comp.neutralized_2d_inchi:
        comp.neutralized_2d_inchi = flatten_inchi(norm_mol)  # type: ignore
    if not comp.neutralized_2d_inchi_key:
        comp.neutralized_2d_inchi_key = Chem.inchi.InchiToInchiKey(comp.neutralized_2d_inchi)
    comp.inchi = Chem.MolToInchi(norm_mol)
    comp.inchi_key = comp.inchi_key or Chem.inchi.InchiToInchiKey(comp.inchi)
    comp.formula = comp.formula or Chem.rdMolDescriptors.CalcMolFormula(norm_mol)
    comp.mono_isotopic_molecular_weight = comp.mono_isotopic_molecular_weight or ExactMolWt(norm_mol)
    comp.permanent_charge = comp.permanent_charge or Chem.GetFormalCharge(norm_mol)
    comp.number_components = comp.number_components or 1  # type: ignore
    comp.num_free_radicals = comp.num_free_radicals or Chem.Descriptors.NumRadicalElectrons(norm_mol)
comp

In [ ]:
df_no_inchi

In [ ]:
iks = df['inchi_key'].tolist()
chunks = [iks[x:x+50] for x in range(0, len(iks), 50)]
temp = []
counter = 0
for chunk_piece in chunks:
    pubchem_compounds = pcp.get_compounds(chunk_piece, "inchikey",)
    for c in pubchem_compounds:
        if len(c.synonyms)>0:
            name = c.synonyms[0]
        else:
            for p in c.record['props']:
                if 'name' in p['urn'].keys():
                    if p['urn']['name']=='Preferred':
                        name = p['value']['sval']
        temp.append({'smiles':c.isomeric_smiles,'inchi':c.inchi,'name':name,'inchi_key':c.inchikey})
    counter += 1
    print(counter)
temp = pd.DataFrame(temp)

In [ ]:

# def chunks(data: List[Any], num: int) -> Generator[List[Any], None, None]:
#     """Yield successive num-sized chunks from data."""
#     for i in range(0, len(data), num):
#         yield data[i : i + num]
        
# def query_pubchem(inchi_key_list, items_per_query=50):
#     out = []
#     counter = 1
#     for inchi_key_sub_list in chunks(inchi_key_list, items_per_query):
#         out.extend(pcp.get_compounds(inchi_key_sub_list, "inchikey"))
#         if counter % 5 == 0:
#             time.sleep(1)
#         counter = counter + 1
#     return out


# def get_pubchem_compound(inchi_key: str, pub_chem_results: List[pcp.Compound]) -> Optional[pcp.Compound]:
#     for compound in pub_chem_results:
#         if compound.inchikey == inchi_key:
#             return compound
#     return None


In [ ]:
temp

In [ ]:
# df = pd.read_csv('/project/projectdirs/metatlas/projects/spectral_libraries/msmsrefs_nist20only.tab',sep='\t',usecols=['name','smiles','inchi_key','inchi','formula','exact_mass'])
# print(df.shape[0])
# df.drop_duplicates('inchi_key',inplace=True)
# print(df.shape[0])
# df.dropna(subset=['inchi_key'],inplace=True)
# print(df.shape[0])
# print(df.shape[0])
# df.head()

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

# import time
# t0 = time.time()
# compound_list = []
counter = 0
for i,row in df.iterrows():
        compound = metob.Compound()
        compound.name = strip_non_ascii(row['name'])#, "utf-8",errors='ignore'))
    #     compound.description = unicode(df_row.Definition, "utf-8",errors='ignore')
    #     compound.synonyms = strip_non_ascii(df_row.synonyms)#, "utf-8",errors='ignore'))
        mol = Chem.MolFromInchi(row.inchi)
        
        formula = rdMolDescriptors.CalcMolFormula(mol)
        mw = Descriptors.ExactMolWt(mol)
        charge = Chem.GetFormalCharge(mol)
        free_radicals = Descriptors.NumRadicalElectrons(mol)

        compound.formula = formula
        compound.mono_isotopic_molecular_weight = mw
        compound.permanent_charge = charge
    #     try:
        compound.num_free_radicals = int(float(free_radicals))
    #     except:
    #         compound.num_free_radicals = int(float(max(df_row.num_radicals.split('///'))))
    # #     compound.number_components = int(1)
        ik = Chem.InchiToInchiKey(row.inchi)
        
        compound.inchi = row.inchi
        compound.inchi_key = ik
    #     compound.neutralized_2d_inchi = df_row.flat_inchi
    #     compound.neutralized_2d_inchi_key = df_row.flat_inchikey
    #     compound.neutralized_inchi = df_row.metatlas_inchi
    #     compound.neutralized_inchi_key = df_row.metatlas_inchi_key

        compound.source = 'nist20'#str(df_row.source_database)
    #     for url_index,database in enumerate(database_list):
    #         if getattr(df_row,database):
    #             setattr(compound, database, str(getattr(df_row,database)) )
    #             if url_list[url_index]:
    #                 ids = str(getattr(df_row,database)).split('///')
    #                 urls = []
    #                 for my_id in ids:
    #                     urls.append('%s%s'%(url_list[url_index],my_id))
    #                     if 'pubchem' in database:
    #                         setattr(compound, database.replace('_compound_id','_url'),'///'.join(urls) )
    #                     else:
    #                         setattr(compound, database.replace('_id','_url'),'///'.join(urls) )
#         compound_list.append(compound)
#         if ik!=row['inchi_key']:
#             print(i)
        metob.store(compound)
        counter += 1
        if counter == 100:
            print(i)
            counter = 0
# print time.time() - t0

In [ ]:
old_cpds = pd.read_csv('/project/projectdirs/metatlas/projects/unique_compounds.csv.gz')
old_cpds.head()

In [ ]:
# df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
# df = pd.read_csv('/global/homes/b/bpb/Downloads/NonMetAtlas-ENZO.csv')
# df.rename(columns={'std_inchi_key':'metatlas_inchi_key',
#                   'std_inchi':'metatlas_inchi',
#                   'name':'common_name',
#                   'source':'source_database'},inplace=True)
cpds = [{'metatlas_inchi_key':'MUYNUOXIHSNESP-UHFFFAOYSA-N',
      'metatlas_inchi':'InChI=1S/C12H12O12/c13-3-1-19-11-5(3)21-7(15)9(11,17)23-10(18)8(16)22-6-4(14)2-20-12(6,10)24-11/h3-6,13-14,17-18H,1-2H2',
      'common_name':'Dehydroascorbic Acid Dimer',
      'source_database':'ChEBI'}]
df = pd.DataFrame(cpds)
df.tail()

In [ ]:
sum(df.metatlas_inchi_key.isin(old_cpds.inchi_key))

In [ ]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

import time
t0 = time.time()
# compound_list = []
counter = 0
for i,df_row in df.iterrows():
    if not df_row.metatlas_inchi_key in old_cpds.inchi_key:
        compound = metob.Compound()
        compound.name = strip_non_ascii(df_row.common_name)#, "utf-8",errors='ignore'))
    #     compound.description = unicode(df_row.Definition, "utf-8",errors='ignore')
    #     compound.synonyms = strip_non_ascii(df_row.synonyms)#, "utf-8",errors='ignore'))
        mol = Chem.MolFromInchi(df_row.metatlas_inchi)
        
        formula = rdMolDescriptors.CalcMolFormula(mol)
        mw = Descriptors.ExactMolWt(mol)
        charge = Chem.GetFormalCharge(mol)
        free_radicals = Descriptors.NumRadicalElectrons(mol)

        compound.formula = formula
        compound.mono_isotopic_molecular_weight = mw
        compound.permanent_charge = charge
    #     try:
        compound.num_free_radicals = int(float(free_radicals))
    #     except:
    #         compound.num_free_radicals = int(float(max(df_row.num_radicals.split('///'))))
    # #     compound.number_components = int(1)

        compound.inchi = df_row.metatlas_inchi
        compound.inchi_key = df_row.metatlas_inchi_key
    #     compound.neutralized_2d_inchi = df_row.flat_inchi
    #     compound.neutralized_2d_inchi_key = df_row.flat_inchikey
    #     compound.neutralized_inchi = df_row.metatlas_inchi
    #     compound.neutralized_inchi_key = df_row.metatlas_inchi_key

        compound.source = str(df_row.source_database)
    #     for url_index,database in enumerate(database_list):
    #         if getattr(df_row,database):
    #             setattr(compound, database, str(getattr(df_row,database)) )
    #             if url_list[url_index]:
    #                 ids = str(getattr(df_row,database)).split('///')
    #                 urls = []
    #                 for my_id in ids:
    #                     urls.append('%s%s'%(url_list[url_index],my_id))
    #                     if 'pubchem' in database:
    #                         setattr(compound, database.replace('_compound_id','_url'),'///'.join(urls) )
    #                     else:
    #                         setattr(compound, database.replace('_id','_url'),'///'.join(urls) )
    # #     compound_list.append(compound)
        metob.store(compound)
        counter += 1
        if counter == 100:
            print i
            counter = 0
print time.time() - t0

In [ ]:
compound

In [ ]:
df.shape

In [ ]:
database_list = ['chebi_id',
                 'hmdb_id',
                 'img_abc_id',
                 'kegg_id',
                 'lipidmaps_id',
                 'metacyc_id',
                 'pubchem_compound_id']
url_list = [
    'http://www.ebi.ac.uk/chebi/searchId.do?chebiId=',
    'http://www.hmdb.ca/metabolites/',
    '',
    'http://www.genome.jp/dbget-bin/www_bget?',
    'http://www.lipidmaps.org/data/LMSDRecord.php?LMID=',
    '',
    'http://pubchem.ncbi.nlm.nih.gov/compound/'
]

In [ ]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

import time
t0 = time.time()
# compound_list = []
counter = 0
for i,df_row in df.iterrows():
    compound = metob.Compound()
    compound.name = strip_non_ascii(df_row.common_name)#, "utf-8",errors='ignore'))
    compound.description = unicode(df_row.Definition, "utf-8",errors='ignore')
    compound.synonyms = strip_non_ascii(df_row.synonyms)#, "utf-8",errors='ignore'))
    
    compound.formula = str(df_row.formula)
    compound.mono_isotopic_molecular_weight = df_row.monoisotopic_mw
    compound.permanent_charge = int(float(df_row.charge))
    try:
        compound.num_free_radicals = int(float(df_row.num_radicals))
    except:
        compound.num_free_radicals = int(float(max(df_row.num_radicals.split('///'))))
    compound.number_components = int(1)
    
    compound.inchi = df_row.metatlas_inchi
    compound.inchi_key = df_row.metatlas_inchi_key
    compound.neutralized_2d_inchi = df_row.flat_inchi
    compound.neutralized_2d_inchi_key = df_row.flat_inchikey
    compound.neutralized_inchi = df_row.metatlas_inchi
    compound.neutralized_inchi_key = df_row.metatlas_inchi_key
    
    compound.source = str(df_row.source_database)
    for url_index,database in enumerate(database_list):
        if getattr(df_row,database):
            setattr(compound, database, str(getattr(df_row,database)) )
            if url_list[url_index]:
                ids = str(getattr(df_row,database)).split('///')
                urls = []
                for my_id in ids:
                    urls.append('%s%s'%(url_list[url_index],my_id))
                    if 'pubchem' in database:
                        setattr(compound, database.replace('_compound_id','_url'),'///'.join(urls) )
                    else:
                        setattr(compound, database.replace('_id','_url'),'///'.join(urls) )
#     compound_list.append(compound)
    metob.store(compound)
    counter += 1
    if counter == 1000:
        print i
        counter = 0
print time.time() - t0

In [ ]:
t0 = time.time()
for c in compound_list:
    metob.store(c)
print time.time() - t0

In [ ]:
14.003074*2+2*e

In [ ]:
# C8H8NO3
e = 0.000548579909
print 12.0*8 + 1.007825*8+14.003074+15.994915*3
print 12.0*8 + 1.007825*8+14.003074+15.994915*3 - e
print 166.049869536

In [ ]:
df[(df.charge==1) & (df.monoisotopoic_mw < 300)].head()

In [ ]:
df.tail(100)

In [ ]:
compound

In [ ]:
compound_list[0].reference_xrefs[0].name

In [ ]:
metatlas_compounds = metob.retrieve('Compounds',name = '%',username='*')

In [ ]:
print len(metatlas_compounds)

In [ ]:
metatlas_compounds[0]

In [ ]:
# metob.FunctionalSet()
# metob.to_dataframe(metatlas_compounds[:1])
mc1 = metob.Compound()
dbr = metob.ReferenceDatabase()
# mc1.reference_xrefs = [dbr,dbr]
print mc1()

In [ ]:
mc1()

In [ ]:
counter = 0
field_list = ['inchi','name']
metatlas_df = pd.DataFrame()
for compound in metatlas_compounds:
    for field in field_list:
        metatlas_df.loc[counter,field] = getattr(compound,field)
    counter += 1

In [ ]:
metatlas_df.head()

In [ ]:
def make_mol_from_smiles_and_inchi(row):
    mol= []
    if row['inchi']:
        mol = Chem.MolFromInchi(row['inchi'].encode('utf-8'))
    if mol:
        return mol

metatlas_df.ROMol = metatlas_df.apply(make_mol_from_smiles_and_inchi, axis=1)

In [ ]:
def desalt_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()>1:
            c = import_compounds.desalt(x)
            if c[1]:
                return c[0]
            else:
                return x
    
metatlas_df.ROMol = metatlas_df.ROMol.apply(desalt_compounds_in_dataframe)


In [ ]:
import_compounds = reload(import_compounds)
def neutralize_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()> 0:
            neutral_mol = []
            try:
                c = import_compounds.NeutraliseCharges(x)
                neutral_mol = c[0]
            except:
                pass
            if neutral_mol:
                return neutral_mol
            
metatlas_df.ROMol = metatlas_df.ROMol.apply(neutralize_compounds_in_dataframe)

In [ ]:
def calculate_num_radicals_in_dataframe(x):
    num_radicals = 0.0
    if x:
        num_radicals = Descriptors.NumRadicalElectrons(x)        
    return num_radicals

def calculate_formula_in_dataframe(x):
    formula = ''
    if x:
        formula = rdMolDescriptors.CalcMolFormula(x)
    return formula

def calculate_monoisotopic_mw_in_dataframe(x):
    mw = 0.0
    if x:
        mw = Descriptors.ExactMolWt(x)
    return mw

def calculate_inchi_in_dataframe(x):
    inchi = ''
    if x:
        try:
            inchi = Chem.MolToInchi(x)
        except:
            pass#This fails when can't kekulize mol
    return inchi

def calculate_flattened_inchi_in_dataframe(x):
    flattened_inchi = ''
    if x:
        sm = Chem.MolToSmiles(x).replace('@','')
        flattened_rdkit_mol = Chem.MolFromSmiles(sm)
        try:
            flattened_inchi = Chem.MolToInchi(flattened_rdkit_mol)
        except:
            pass#This fails when can't kekulize mol
    return flattened_inchi

def calculate_inchikey_in_dataframe(x):
    ik = ''
    if x:
        try:
            ik = Chem.InchiToInchiKey(x)
        except:
            pass#This fails when can't kekulize mol.  Carbo-cations are the culprit usually. 
    return ik

def calculate_charge_in_dataframe(x):
    if x:
        my_charge = Chem.GetFormalCharge(x)
        return my_charge
            
metatlas_df['charge'] = metatlas_df.ROMol.apply(calculate_charge_in_dataframe)
metatlas_df['formula'] = metatlas_df.ROMol.apply(calculate_formula_in_dataframe)
metatlas_df['monoisotopoic_mw'] = metatlas_df.ROMol.apply(calculate_monoisotopic_mw_in_dataframe)
metatlas_df['num_radicals'] = metatlas_df.ROMol.apply(calculate_num_radicals_in_dataframe)

metatlas_df['metatlas_inchi'] = metatlas_df.ROMol.apply(calculate_inchi_in_dataframe)
metatlas_df['metatlas_inchi_key'] = metatlas_df.metatlas_inchi.apply(calculate_inchikey_in_dataframe)

metatlas_df['flat_inchi'] = metatlas_df.ROMol.apply(calculate_flattened_inchi_in_dataframe)
metatlas_df['flat_inchikey'] = metatlas_df.flat_inchi.apply(calculate_inchikey_in_dataframe)


In [ ]:
metatlas_df = metatlas_df.where((pd.notnull(metatlas_df)), None)


In [ ]:
metatlas_df = metatlas_df[metatlas_df.metatlas_inchi_key != '']


In [ ]:
metatlas_df.to_pickle('backup of all metatlas compounds with flattened inchi and key.pkl')
metatlas_df.head()

In [ ]:
idx = metatlas_df.metatlas_inchi_key.isin(df.metatlas_inchi_key)

In [ ]:
not_found = metatlas_df[~idx]

In [ ]:
not_found = not_found.drop_duplicates()

In [ ]:
not_found.head(20)

In [ ]:
# not_found.to_csv('metatlas_compounds_not_found.csv')
# not_found.to_pickle('metatlas_compounds_not_found.pkl')

In [ ]:
not_found.shape